In [1]:
import numpy as np 
import cv2 as cv 
from os.path import join
import os
import matplotlib.pyplot as plt 
from scipy.interpolate import RectBivariateSpline
import open3d as o3d
%matplotlib qt

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [15]:
#path of the images
path = r'D:\Dalhousie\datasets\Registration'
folder = r'Series1_crop' 

saving_folder = r'Results'

in_path=join(path,folder)
entries = os.listdir(in_path)
image_path = join(in_path,entries[0]) #Selected image
savedimage_path = join(path,saving_folder)

# B-Spline Surface

## Saving volume

In [3]:
I = cv.imread(image_path,0)
vol = np.zeros((I.shape[0],I.shape[1],len(entries)),np.uint8)

print(len(entries))

for ith in range (len(entries)):    
    image_path = join(in_path,entries[ith])
    I = cv.imread(image_path,0)
    vol[:,:,ith] = I

npz_file = 'vol.npz'
print(f"Saving {npz_file}")
np.savez(npz_file, vol=vol)
print("Done!")

vol[vol<60] = 0

y,x,z = np.where(vol)

points = np.vstack((x, y, z)).T

coordinates = points

min_vals = np.min(coordinates, axis=0)
max_vals = np.max(coordinates, axis=0)

print(min_vals,max_vals)

# Normalize the coordinates to range between -1 and 1
normalized_coordinates = (coordinates - min_vals) / (max_vals - min_vals)

print("Normalized coordinates:")
print(np.max(normalized_coordinates, axis=0), np.min(normalized_coordinates, axis=0))



pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[::10,:])
pcd.paint_uniform_color([1, 0.706, 0])
o3d.visualization.draw_geometries([pcd])

o3d.io.write_point_cloud("Series2.pcd", pcd)

510
Saving vol.npz
Done!
[  3 416   0] [855 599 509]
Normalized coordinates:
[1. 1. 1.] [0. 0. 0.]


True

## Bivarriate Spline Fitting 

In [7]:
# Load volume from disk.  Run save_vol.py to generate the npz file with
# the volume data
vol=np.load("vol.npz")['vol']
vol=vol[300:,:,:]
print(vol.shape)
#%%
# Make point cloud of bone contour
points=np.argmax(vol, axis=0).T
points=points.astype('float64')


mask = (points>30)


# Set up normalized axis arrays
Nx, Ny, Nz = vol.shape
x=np.linspace(-1,1,Nx)
y=np.linspace(-1,1,Ny)
z=np.linspace(-1,1,Nz)
Y,Z = np.meshgrid(y,z)



order = 3
# Do the spline fit
rbs = RectBivariateSpline(y,z,points.T, bbox=[-1,1,-1,1], kx=order, ky=order, s=9e7)
# Get smoothed surface location estimates
xfit=rbs.ev(Y,Z)


from scipy.ndimage import binary_erosion
fig = plt.figure()
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
ax2 = fig.add_subplot(1, 2, 2, projection='3d')

# Mask out point outside of botn surface
points[np.logical_not(mask)] = np.nan
ax1.plot_surface(Y, Z, points)
ax1.set_title("Raw data")

# Spline fit produces artefact around the edges so we need to erode
# the mask inward to avoid displaying them
mask2=binary_erosion(mask, iterations=10)
xfit[np.logical_not(mask2)] = np.nan 
ax2.plot_surface(Y, Z, xfit)
ax2.set_title("Spline fit")
plt.show()

(366, 856, 510)


# Mesh-based 3D Model

## Marching Cubes algorithm from skimage 

In [16]:
import numpy as np
import cv2 as cv 
from skimage import measure, morphology
from stl import mesh

I=cv.imread(image_path,0) #Reading an image to obtain shape 

# Array where we merge all the segmentation masks from the series 
Array_3D = np.zeros((I.shape[0],I.shape[1],len(entries)),np.uint8)

#Merging process
for ith in range (len(entries)):
    image_path = join(in_path,entries[ith])
    I = cv.imread(image_path,0)
    Array_3D[:,:,ith] = I
    
binary_array = Array_3D > 0

verts, faces, _, _ = measure.marching_cubes(binary_array, 0) #Marchin cubes algorithm 

# Create a mesh object
surface_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))

# Assign vertices and faces to the mesh
for i, face in enumerate(faces):
    for j in range(3):
        surface_mesh.vectors[i][j] = verts[face[j]]

# Save the mesh to an STL file
surface_mesh.save('SphereMeshModel.stl')

ya fue
(666, 853, 510)
ya merito
done


# Point Cloud 3D Model

In [17]:
import numpy as np 
import cv2 as cv 
from os.path import join
import os
import matplotlib.pyplot as plt 
from scipy.interpolate import RectBivariateSpline
import open3d as o3d
%matplotlib qt

I = cv.imread(image_path,0)
 
# Array where we merge all the segmentation masks from the series
Array_3D = np.zeros((I.shape[0],I.shape[1],len(entries)),np.uint8)  


#Merging process
for ith in range (len(entries)):    
    image_path = join(in_path,entries[ith])
    I = cv.imread(image_path,0)
    Array_3D[:,:,ith] = I


#Saving merge volume    
npz_file = 'vol.npz'
print(f"Saving {npz_file}")
np.savez(npz_file, vol=Array_3D)
print("Done!")


Array_3D[Array_3D<60] = 0

y,x,z = np.where(Array_3D)

points = np.vstack((x, y, z)).T

coordinates = points

min_vals = np.min(coordinates, axis=0)
max_vals = np.max(coordinates, axis=0)


# Normalize the coordinates to range between -1 and 1
normalized_coordinates = (coordinates - min_vals) / (max_vals - min_vals)

print("Normalized coordinates:")
print(np.max(normalized_coordinates, axis=0), np.min(normalized_coordinates, axis=0))



pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points[::10,:])
pcd.paint_uniform_color([1, 0.706, 0])
o3d.visualization.draw_geometries([pcd])

o3d.io.write_point_cloud("Series2.pcd", pcd)

Saving vol.npz
Done!
Normalized coordinates:
[1. 1. 1.] [0. 0. 0.]


True